# Configurando e criando sessao spark

In [2]:
#Configuração variavel de ambiente
import os

os.environ['SPARK_HOME'] = 'C:\spark-3.5.3-bin-hadoop3' 

In [3]:
#Encontrar o spark na maquina

import findspark
findspark.init()

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [5]:
#Criar a sessao no spark session

spark = SparkSession.builder\
                    .appName('Praticando-Pyspark')\
                    .master('local[*]')\
                    .getOrCreate()

In [6]:
spark

# Carregando e conhecendo os datasets

In [7]:
df_pedidos = spark.read.csv('orders_dataset.csv', header = True, inferSchema = True)
df_pagamentos = spark.read.csv('payments_dataset.csv', header = True, inferSchema = True)
df_produtos = spark.read.csv('products_dataset.csv', header = True, inferSchema = True)
df_sellers = spark.read.csv('sellers_dataset.csv', header = True, inferSchema = True)
df_clientes = spark.read.csv('customers_dataset.csv', header = True, inferSchema = True)
df_itens_pedido = spark.read.csv('order_items_dataset.csv', header = True, inferSchema = True)

In [8]:
# Conferindo o tipo de dados de cada dataset
datasets = [
    ("Pedidos", df_pedidos),
    ("Pagamentos", df_pagamentos),
    ("Produtos", df_produtos),
    ("Sellers", df_sellers),
    ("Clientes", df_clientes),
    ("itens_do_pedido",df_itens_pedido)
]

for name, df in datasets:
    print(f"Esquema do DataFrame: {name}")
    df.printSchema()
    print("-" * 40)


Esquema do DataFrame: Pedidos
root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)

----------------------------------------
Esquema do DataFrame: Pagamentos
root
 |-- order_id: string (nullable = true)
 |-- payment_sequential: integer (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- payment_installments: integer (nullable = true)
 |-- payment_value: double (nullable = true)

----------------------------------------
Esquema do DataFrame: Produtos
root
 |-- product_id: string (nullable = true)
 |-- product_category_name: string (nullable = true)
 |-- product_name_lenght: integer (nullable = true

In [9]:
df_pedidos.toPandas().isnull().sum()

order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64

In [10]:
# Renomear colunas 

df_pedidos = df_pedidos.withColumnRenamed('order_id','id_order')\
.withColumnRenamed('customer_id','id_customer').withColumnRenamed('order_status','status').withColumnRenamed('order_purchase_timestamp','order_purchase_time')

# Filtros

In [11]:
#Filtros

# Filtro simples
df_produtos.filter("product_category_name == 'brinquedos'").show(5)

+--------------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+
|          product_id|product_category_name|product_name_lenght|product_description_lenght|product_photos_qty|product_weight_g|product_length_cm|product_height_cm|product_width_cm|
+--------------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+
|8c92109888e8cdf9d...|           brinquedos|                 36|                      1156|                 1|             600|               17|               10|              12|
|1c6fb703c624b381a...|           brinquedos|                 33|                       471|                 2|             725|               22|               17|              15|
|83b9bc6aae6f527ff...|           brinquedos|                 20|                       866|    

In [12]:
# Filtro com e

df_produtos.filter((col('product_category_name') == 'brinquedos') & (col('product_weight_g') > 500)).show(5)

+--------------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+
|          product_id|product_category_name|product_name_lenght|product_description_lenght|product_photos_qty|product_weight_g|product_length_cm|product_height_cm|product_width_cm|
+--------------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+
|8c92109888e8cdf9d...|           brinquedos|                 36|                      1156|                 1|             600|               17|               10|              12|
|1c6fb703c624b381a...|           brinquedos|                 33|                       471|                 2|             725|               22|               17|              15|
|83b9bc6aae6f527ff...|           brinquedos|                 20|                       866|    

In [13]:
# Filtro com ou

df_produtos.filter((col('product_category_name') == 'brinquedos') | (col('product_category_name') == 'perfumaria')).show(5)

+--------------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+
|          product_id|product_category_name|product_name_lenght|product_description_lenght|product_photos_qty|product_weight_g|product_length_cm|product_height_cm|product_width_cm|
+--------------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+
|1e9e8ef04dbcff454...|           perfumaria|                 40|                       287|                 1|             225|               16|               10|              14|
|8c92109888e8cdf9d...|           brinquedos|                 36|                      1156|                 1|             600|               17|               10|              12|
|6a2fb4dd53d2cdb88...|           perfumaria|                 39|                       346|    

# Criar colunas

In [14]:
# Criando colunas

#Extraindo o ano da compra
df_pedidos = df_pedidos.withColumn('ano_compra',substring('order_purchase_time',1,4))

df_pedidos.select('ano_compra').show(4)

+----------+
|ano_compra|
+----------+
|      2017|
|      2018|
|      2018|
|      2017|
+----------+
only showing top 4 rows



In [15]:
from pyspark.sql.types import *
from pyspark.sql import functions as sf

#Extraindo a data da compra 
df_pedidos = df_pedidos.withColumn('data_compra',col('order_purchase_time').cast(DateType()))

#Entendendo o tempo da compra até o pedido chegar
df_pedidos = df_pedidos.withColumn('dias_compra_ate_chegar',datediff(col('order_delivered_customer_date'),col('order_purchase_time')))



In [16]:
df_pedidos.select('data_compra','dias_compra_ate_chegar').show(4)

+-----------+----------------------+
|data_compra|dias_compra_ate_chegar|
+-----------+----------------------+
| 2017-10-02|                     8|
| 2018-07-24|                    14|
| 2018-08-08|                     9|
| 2017-11-18|                    14|
+-----------+----------------------+
only showing top 4 rows



# EDA

In [17]:
# Quantidade de categorias de produtos

df_produtos.select('product_category_name').distinct().count()

74

In [18]:
df_pedidos_pagamentos = df_pedidos.join(df_pagamentos, df_pedidos['id_order'] == df_pagamentos['order_id'], 'inner')

In [19]:
#Metodos de pagamentos com maior representatividade em valor

df_pedidos_pagamentos.groupby('payment_type').agg(sum(col('payment_value')).alias('soma_valor_por_tipo_de_pagamento')).orderBy('soma_valor_por_tipo_de_pagamento', ascending = False).show()

+------------+--------------------------------+
|payment_type|soma_valor_por_tipo_de_pagamento|
+------------+--------------------------------+
| credit_card|            1.2542084189999647E7|
|      boleto|              2869361.2700000196|
|     voucher|               379436.8700000001|
|  debit_card|              217989.79000000015|
| not_defined|                             0.0|
+------------+--------------------------------+



In [20]:
#Ticket medio por metodo de pagamento

df_pedidos_pagamentos.groupby('payment_type').agg((sum('payment_value') / count('id_order')).alias('tkm_por_metodo_de_pagamento')).orderBy('tkm_por_metodo_de_pagamento', ascending = False).show()

+------------+---------------------------+
|payment_type|tkm_por_metodo_de_pagamento|
+------------+---------------------------+
| credit_card|         163.31902063935996|
|      boleto|         145.03443540234633|
|  debit_card|         142.57017004578165|
|     voucher|          65.70335411255414|
| not_defined|                        0.0|
+------------+---------------------------+



In [21]:
# Quantos clientes fizeram pedidos ?

df_pedidos.filter(df_pedidos.id_customer.isNotNull()).select('id_customer').distinct().count()

99441

In [22]:
# Qual a distribuição geografica dos clientes

df_clientes_pedidos = df_pedidos_pagamentos.join(df_clientes,  df_clientes['customer_id'] == df_pedidos_pagamentos['id_customer'], 'left') 

In [23]:
df_clientes_pedidos.groupby('customer_city').count().orderBy('count', ascending=False).show()

+--------------------+-----+
|       customer_city|count|
+--------------------+-----+
|           sao paulo|16221|
|      rio de janeiro| 7207|
|      belo horizonte| 2872|
|            brasilia| 2193|
|            curitiba| 1576|
|            campinas| 1515|
|        porto alegre| 1418|
|            salvador| 1347|
|           guarulhos| 1250|
|sao bernardo do c...|  979|
|             niteroi|  915|
|         santo andre|  820|
|              osasco|  783|
|             goiania|  741|
|              santos|  733|
| sao jose dos campos|  729|
|           fortaleza|  683|
|            sorocaba|  657|
|              recife|  639|
|             jundiai|  593|
+--------------------+-----+
only showing top 20 rows



In [24]:
# Taxa de clientes que repetiram a compra

df_clientes_pedidos.filter(df_clientes_pedidos.id_order.isNotNull()).select('id_order').count()

103886

In [25]:
# Taxa de pedidos por clientes

df_clientes_pedidos.filter(df_clientes_pedidos.id_order.isNotNull()).select('id_order').count()  /  df_pedidos.filter(df_pedidos.id_customer.isNotNull()).select('id_customer').distinct().count()

1.0446998722860792

In [26]:
# Os produtos mais vendidos

df_produtos  = df_produtos.withColumnRenamed('product_id','id_produto')
df_pedidos_produtos = df_itens_pedido.join(df_produtos, df_itens_pedido['product_id'] == df_produtos['id_produto'], 'left')

In [27]:
df_pedidos_produtos.groupby('product_id','product_category_name').count().orderBy('count', ascending=False).show()

+--------------------+---------------------+-----+
|          product_id|product_category_name|count|
+--------------------+---------------------+-----+
|aca2eb7d00ea1a7b8...|     moveis_decoracao|  527|
|99a4788cb24856965...|      cama_mesa_banho|  488|
|422879e10f4668299...|   ferramentas_jardim|  484|
|389d119b48cf3043d...|   ferramentas_jardim|  392|
|368c6c730842d7801...|   ferramentas_jardim|  388|
|53759a2ecddad2bb8...|   ferramentas_jardim|  373|
|d1c427060a0f73f6b...| informatica_acess...|  343|
|53b36df67ebb7c415...|   relogios_presentes|  323|
|154e7e31ebfa09220...|         beleza_saude|  281|
|3dd2a17168ec895c7...| informatica_acess...|  274|
|2b4609f8948be1887...|         beleza_saude|  260|
|7c1bd920dbdf22470...|         beleza_saude|  231|
|a62e25e09e05e6faf...|   relogios_presentes|  226|
|5a848e4ab52fd5445...|                 NULL|  197|
|bb50f2e236e5eea01...|         beleza_saude|  195|
|e0d64dcfaa3b6db5c...|   relogios_presentes|  194|
|42a2c92a0979a949c...| utilidad

In [28]:
df_clientes_pedidos.columns

['id_order',
 'id_customer',
 'status',
 'order_purchase_time',
 'order_approved_at',
 'order_delivered_carrier_date',
 'order_delivered_customer_date',
 'order_estimated_delivery_date',
 'ano_compra',
 'data_compra',
 'dias_compra_ate_chegar',
 'order_id',
 'payment_sequential',
 'payment_type',
 'payment_installments',
 'payment_value',
 'customer_id',
 'customer_unique_id',
 'customer_zip_code_prefix',
 'customer_city',
 'customer_state']

In [29]:
# Quantidade de pedidos por quantidade de fotos

df_pedidos_produtos.groupby('product_photos_qty').agg(count(col('order_id')).alias('Quantidade de pedidos')).orderBy('Quantidade de pedidos', ascending = False).show()

+------------------+---------------------+
|product_photos_qty|Quantidade de pedidos|
+------------------+---------------------+
|                 1|                56028|
|                 2|                21963|
|                 3|                12392|
|                 4|                 8437|
|                 5|                 5368|
|                 6|                 3786|
|              NULL|                 1603|
|                 7|                 1501|
|                 8|                  727|
|                10|                  342|
|                 9|                  313|
|                11|                   71|
|                12|                   53|
|                13|                   30|
|                15|                   12|
|                17|                   11|
|                14|                    6|
|                18|                    4|
|                19|                    2|
|                20|                    1|
+----------

In [30]:
#  Valor dos pedidos por quantidade de fotos

df_pedidos_produtos.groupby('product_photos_qty').agg((sum(col('price'))).alias('valor_total_por_qtd_fotos')).orderBy('valor_total_por_qtd_fotos', ascending = False).show()

+------------------+-------------------------+
|product_photos_qty|valor_total_por_qtd_fotos|
+------------------+-------------------------+
|                 1|       6371606.6199994655|
|                 2|        2449270.980000071|
|                 3|       1683253.4800000067|
|                 4|       1228562.2599999951|
|                 5|        728390.6399999927|
|                 6|        499489.7499999988|
|                 7|       238081.86000000016|
|              NULL|       179535.28000000032|
|                 8|       108374.97000000004|
|                10|        51516.76999999996|
|                 9|       40301.869999999966|
|                11|                  5342.27|
|                12|       2551.9799999999996|
|                14|                  1500.87|
|                13|                  1236.78|
|                15|                  1043.77|
|                17|                    841.0|
|                18|                    354.4|
|            

In [31]:
# Ticket medio dos pedidos por quantidade de fotos
df_pedidos_produtos.groupby('product_photos_qty').agg((sum(col('price')) / count(col('order_id'))).alias('valor_total_por_qtd_fotos')).orderBy('valor_total_por_qtd_fotos', ascending = False).show()

+------------------+-------------------------+
|product_photos_qty|valor_total_por_qtd_fotos|
+------------------+-------------------------+
|                14|       250.14499999999998|
|                 7|       158.61549633577624|
|                10|       150.63383040935662|
|                 8|       149.07148555708397|
|                 4|        145.6160080597363|
|                19|                   138.94|
|                 3|       135.83388315042018|
|                 5|       135.69125186288986|
|                 6|       131.93073164289456|
|                 9|        128.7599680511181|
|                 1|       113.72182872848336|
|              NULL|       111.99955084217113|
|                 2|       111.51805217866736|
|                20|                   110.27|
|                18|                     88.6|
|                15|        86.98083333333334|
|                17|        76.45454545454545|
|                11|        75.24323943661973|
|            

In [32]:
#Criando coluna do valor total (Preco dos itens + frete)

df_pedidos_produtos = df_pedidos_produtos.withColumn('valor_total', col('price') + col('freight_value'))

In [33]:
df_pedidos_produtos.select(col('order_id'),col('price'), col('freight_value'), col('valor_total')).show()

+--------------------+------+-------------+------------------+
|            order_id| price|freight_value|       valor_total|
+--------------------+------+-------------+------------------+
|00010242fe8c5a6d1...|  58.9|        13.29|             72.19|
|00018f77f2f0320c5...| 239.9|        19.93|            259.83|
|000229ec398224ef6...| 199.0|        17.87|            216.87|
|00024acbcdf0a6daa...| 12.99|        12.79|             25.78|
|00042b26cf59d7ce6...| 199.9|        18.14|218.04000000000002|
|00048cc3ae777c65d...|  21.9|        12.69|34.589999999999996|
|00054e8431b9d7675...|  19.9|        11.85|             31.75|
|000576fe39319847c...| 810.0|        70.75|            880.75|
|0005a1a1728c9d785...|145.95|        11.65|             157.6|
|0005f50442cb953dc...| 53.99|         11.4|             65.39|
|00061f2a7bc09da83...| 59.99|         8.88|             68.87|
|00063b381e2406b52...|  45.0|        12.98|57.980000000000004|
|0006ec9db01a64e59...|  74.0|        23.32|            

In [34]:
# Categorias com maior representatividade de valor de vendas

df_pedidos_produtos.groupby('product_category_name').agg(sum('price').alias('valor_total_venda_produto')).orderBy('valor_total_venda_produto' , ascending = False).show()

+---------------------+-------------------------+
|product_category_name|valor_total_venda_produto|
+---------------------+-------------------------+
|         beleza_saude|       1258681.3399999938|
|   relogios_presentes|        1205005.679999998|
|      cama_mesa_banho|       1036988.6799999807|
|        esporte_lazer|        988048.9699999837|
| informatica_acess...|         911954.319999988|
|     moveis_decoracao|        729762.4899999866|
|           cool_stuff|        635290.8499999974|
| utilidades_domest...|        632248.6599999928|
|           automotivo|         592720.109999997|
|   ferramentas_jardim|       485256.45999999647|
|           brinquedos|       483946.59999999864|
|                bebes|       411764.88999999885|
|           perfumaria|        399124.8699999997|
|            telefonia|        323667.5300000011|
|    moveis_escritorio|        273960.7000000001|
|            papelaria|       230943.23000000085|
|                  pcs|                222963.13|


In [39]:
# Estados com mais pedidos 

df_pedidos_por_estados = df_clientes_pedidos.groupby('customer_state').agg(count('id_order').alias('quantidade_de_pedidos')).orderBy('quantidade_de_pedidos' , ascending = False)

In [36]:
nordeste = ["AL", "BA", "CE", "MA", "PB", "PE", "PI", "RN", "SE"]
sudeste = ["ES", "MG", "RJ", "SP"]
norte = ["AC", "AP", "AM", "PA", "RO", "RR", "TO"]
sul = ["PR", "RS", "SC"]
centro_oeste = ["DF", "GO", "MT", "MS"]

In [40]:
df_pedidos_por_estados = df_pedidos_por_estados.withColumn('regiao', when(col('customer_state').isin(nordeste),"Nordeste")\
                            .when(col('customer_state').isin(sul),"Sul")\
                            .when(col('customer_state').isin(norte),"Norte")\
                            .when(col('customer_state').isin(centro_oeste),"Centro-oeste")\
                            .when(col('customer_state').isin(sudeste),"Sudeste")\
                            .otherwise("Desconhecido"))

In [41]:
# Regioes com mais pedidos

df_pedidos_por_estados.groupby('regiao').agg(sum('quantidade_de_pedidos').alias('geral_por_regiao')).orderBy('geral_por_regiao', ascending = False).show()

+------------+----------------+
|      regiao|geral_por_regiao|
+------------+----------------+
|     Sudeste|           71358|
|         Sul|           14684|
|    Nordeste|            9907|
|Centro-oeste|            6010|
|       Norte|            1927|
+------------+----------------+

